<a href="https://colab.research.google.com/github/kmeng01/rome/blob/main/notebooks/causal_trace.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

In [1]:
IS_COLAB = False

## Causal Tracing

A demonstration of the double-intervention causal tracing method.

The strategy used by causal tracing is to understand important
states within a transfomer by doing two interventions simultaneously:

1. Corrupt a subset of the input.  In our paper, we corrupt the subject tokens
   to frustrate the ability of the transformer to accurately complete factual
   prompts about the subject.
2. Restore a subset of the internal hidden states.  In our paper, we scan
   hidden states at all layers and all tokens, searching for individual states
   that carry the necessary information for the transformer to recover its
   capability to complete the factual prompt.

The traces of decisive states can be shown on a heatmap.  This notebook
demonstrates the code for conducting causal traces and creating these heatmaps.

In [2]:
%load_ext autoreload
%autoreload 2

The `experiments.causal_trace` module contains a set of functions for running causal traces.

In this notebook, we reproduce, demonstrate and discuss the interesting functions.

We begin by importing several utility functions that deal with tokens and transformer models.

In [3]:
import os, sys, re, json
import string
import torch
import numpy as np
import copy
from collections import defaultdict
from util import nethook
from util.globals import DATA_DIR
from experiments.causal_trace import (
    ModelAndTokenizer,
    layername,
    guess_subject,
    plot_trace_heatmap,
)
from experiments.causal_trace import (
    make_inputs,
    decode_tokens,
    find_token_range,
    predict_token,
    predict_from_input,
    collect_embedding_std,
)
from dsets import KnownsDataset

In [4]:
torch.set_grad_enabled(False)

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## USKG

In [6]:
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)

# from uskg.models.unified.prefixtuning import Model
from uskg.models.unified import finetune, prefixtuning
from uskg.utils.configue import Configure
from uskg.utils.training_arguments import WrappedSeq2SeqTrainingArguments
from uskg.seq2seq_construction import spider as s2s_spider
from uskg.third_party.spider.preprocess.get_tables import dump_db_json_schema
from uskg.third_party.spider import evaluation as sp_eval
from tqdm.auto import tqdm

# from nltk.stem.wordnet import WordNetLemmatizer
# import stanza

import matplotlib.pyplot as plt
import sqlite3

from experiments import causal_trace_uskg as ctu

In [7]:
mt_uskg = ctu.ModelAndTokenizer_USKG('t5-large-prefix')

Using tokenizer_uskg: hkunlp/from_all_T5_large_prefix_spider_with_cell_value2
Using tokenizer_fast: t5-large
prefix-tuning sequence length is 10.


In [8]:
list(mt_uskg.task_args.seq2seq)

[('constructor', 'seq2seq_construction.spider'),
 ('schema_serialization_with_db_content', True),
 ('target_with_db_id', False)]

In [9]:
mt_uskg.model.pretrain_model.encoder.embed_tokens is mt_uskg.model.pretrain_model.shared, \
mt_uskg.model.pretrain_model.decoder.embed_tokens is mt_uskg.model.pretrain_model.shared

(True, False)

In [10]:
# [k for k,v in mt_uskg.model.named_parameters()]

In [11]:
inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    enc_sentences=["Translate to German: My name is Wolfgang and I live in Berlin"],
    dec_prompts=["Mein Name ist Wolfgang"],
    device="cuda:0"
)

In [12]:
out = ctu.run_model_forward_uskg(mt_uskg.model, **inp)

In [13]:
out.keys(), out['logits'].size()

(odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state']),
 torch.Size([1, 5, 32102]))

In [14]:
logits = out["logits"][0, -1].detach().cpu().numpy()
logits.shape

(32102,)

In [15]:
top_5 = sorted(list(enumerate(logits)), key=lambda p: -p[1])[:5]
top_5

[(11, -1.8642352),
 (6, -9.727753),
 (5, -10.966707),
 (27, -11.037394),
 (213, -12.864212)]

In [16]:
[mt_uskg.tokenizer.decode([p[0]]) for p in top_5]

['and', ',', '.', 'I', 'where']

### Load spider dataset

In [142]:
spider_train_path = '/home/yshao/Projects/SDR-analysis/data/spider/train+ratsql_graph.json'
spider_dev_path = '/home/yshao/Projects/SDR-analysis/data/spider/dev+ratsql_graph.json'
spider_db_dir = '/home/yshao/Projects/language/language/xsp/data/spider/database'

In [18]:
raw_spider_dev = ctu.load_raw_dataset(
    data_filepath = spider_dev_path,
    db_path=spider_db_dir,
#     schema_cache=SCHEMA_CACHE
)
len(raw_spider_dev)

1034

In [19]:
raw_spider_dev[0].keys()

dict_keys(['query', 'question', 'db_id', 'db_path', 'db_table_names', 'db_column_names', 'db_column_types', 'db_primary_keys', 'db_foreign_keys', 'rat_sql_graph'])

In [20]:
mt_uskg.task_args.dataset.use_cache

True

In [21]:
processed_spider_dev = s2s_spider.DevDataset(
    args=mt_uskg.task_args,
    raw_datasets=raw_spider_dev,
    cache_root='../cache')

In [22]:
processed_spider_dev[4]['text_in'], \
processed_spider_dev[4]['struct_in'], \
processed_spider_dev[4]['seq_out']

('What is the average, minimum, and maximum age of all singers from France?',
 '| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id',
 "select avg(age), min(age), max(age) from singer where country = 'France'")

In [ ]:
# _occ_punct = set()

for _id in range(len(processed_spider_dev)):
    ex = processed_spider_dev[_id]
#     _occ_punct.update(set(string.punctuation) & set(ex['seq_out']))
    if '_(' in ex['struct_in']:
        print(_id, ex['question'])
        print(ex['struct_in'])
        print(ex['seq_out'])
        print()

In [143]:
## Train set

raw_spider_train = ctu.load_raw_dataset(
    data_filepath = spider_train_path,
    db_path=spider_db_dir,
)
processed_spider_train = s2s_spider.TrainDataset(
    args=mt_uskg.task_args,
    raw_datasets=raw_spider_train,
    cache_root='../cache')
len(processed_spider_train)

7000

In [151]:
processed_spider_train[5441]

{'query': "SELECT T2.name FROM membership_register_branch AS T1 JOIN branch AS T2 ON T1.branch_id  =  T2.branch_id JOIN member AS T3 ON T1.member_id  =  T3.member_id WHERE T3.Hometown  =  'Louisville ,  Kentucky' INTERSECT SELECT T2.name FROM membership_register_branch AS T1 JOIN branch AS T2 ON T1.branch_id  =  T2.branch_id JOIN member AS T3 ON T1.member_id  =  T3.member_id WHERE T3.Hometown  =  'Hiram ,  Georgia'",
 'question': 'What are the names of the branches that have some members with a hometown in Louisville, Kentucky and also those from Hiram, Goergia?',
 'db_id': 'shop_membership',
 'db_path': '/home/yshao/Projects/language/language/xsp/data/spider/database',
 'db_table_names': ['member',
  'branch',
  'membership_register_branch',
  'purchase'],
 'db_column_names': {'table_id': [-1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   2,
   2,
   2,
   3,
   3,
   3,
   3],
  'column_name': ['*',
   'Member_ID',
   'Card_Number',
   'Name',
   'Hometown',
  

#### Analysis sample 1 (ID = 0)

In [32]:
ex = processed_spider_dev[0]
ex['question'], ex['seq_out']

('How many singers do we have?', 'select count(*) from singer')

In [33]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
dec_prompt = 'select count(*) from'

inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    enc_sentences=[enc_sentence],
    dec_prompts=[dec_prompt],
    device="cuda:0"
)

In [34]:
mt_uskg.tokenizer.decode(inp['input_ids'][0]), mt_uskg.tokenizer.decode(inp['decoder_input_ids'][0])

('How many singers do we have?; structed knowledge: | concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id</s>',
 '<pad> select count(*) from')

In [35]:
out = ctu.run_model_forward_uskg(mt_uskg.model, **inp)

In [36]:
out.keys(), out['logits'].size()

(odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state']),
 torch.Size([1, 7, 32102]))

In [37]:
logits = out["logits"][0, -1].detach().cpu().numpy()
logits.shape

(32102,)

In [38]:
top_5 = sorted(list(enumerate(logits)), key=lambda p: -p[1])[:5]
top_5

[(7634, -5.629301),
 (6721, -15.1248665),
 (10159, -17.77869),
 (2377, -18.263933),
 (8782, -18.631098)]

In [39]:
[mt_uskg.tokenizer.decode([p[0]]) for p in top_5]

['singer', 'vocal', 'sing', 'artist', 'singing']

In [40]:
# e_range = ctu.find_token_range(mt_uskg.tokenizer, inp["input_ids"][0], 'singer')
# e_range

In [41]:
text_range, struct_range = ctu.find_text_struct_in_range(mt_uskg.tokenizer, inp["input_ids"][0])
text_range, struct_range

((0, 8), (15, 125))

In [42]:
tb, te = text_range
sb, se = struct_range
mt_uskg.tokenizer.decode(inp['input_ids'][0]), \
mt_uskg.tokenizer.decode(inp['input_ids'][0][tb:te]), \
mt_uskg.tokenizer.decode(inp['input_ids'][0][sb:se])

('How many singers do we have?; structed knowledge: | concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id</s>',
 'How many singers do we have?',
 '| concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id')

In [43]:
result = ctu.calculate_hidden_flow_uskg(
    mt_uskg,
    enc_sentence=enc_sentence,
    dec_prompt=dec_prompt,
    subject='singer',
    replace=True,
)

calculate_hidden_flow_uskg(): corrupted input: *How *many *singer *s *do *we *have *? *; * *struct *e *d *knowledge *: *| *concert *_ *s *inger *| *stadium * *: *stadium *_ *i *d * *, *location * *, *name * *, *capacity * *, *highest * *, *lowest * *, *average *| *singer * *: *singer *_ *i *d * *, *name * *, *country * *, *song *_ *name * *, *song *_ *release *_ *year * *, *age * *, *is *_ *male *| *concert * *: *concert *_ *i *d * *, *concert *_ *name * *, *theme * *, *stadium *_ *i *d * *, *year *| *singer *_ *in *_ *conce *r *t * *: *concert *_ *i *d * *, *singer *_ *i *d *</s>


trace_important_states_uskg.encoder:   0%|          | 0/1512 [00:00<?, ?it/s]

trace_important_states_uskg.decoder:   0%|          | 0/84 [00:00<?, ?it/s]

In [44]:
result.keys(), result['correct_prediction']

(dict_keys(['scores', 'low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'answer', 'window', 'correct_prediction', 'kind']),
 True)

In [45]:
result['low_score']

3.2487089356436627e-06

In [ ]:
ctu.plot_trace_heatmap_t5(result)

#### Analysis sample 2 (ID = 9)

In [50]:
_id = 9
ex = processed_spider_dev[_id]
ex['question'], ex['seq_out']

('What are  the different countries with singers above age 20?',
 'select distinct country from singer where age > 20')

In [51]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
dec_prompt = "select distinct country from singer where"
expect = "age"

In [52]:
struct_in

'| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id'

In [53]:
result = ctu.calculate_hidden_flow_uskg(
    mt_uskg,
    enc_sentence=enc_sentence,
    dec_prompt=dec_prompt,
    subject='singer',
    replace=True,
    expect=expect,
)

calculate_hidden_flow_uskg(): corrupted input: *What *are *the *different *countries *with *singer *s *above *age *20 *? *; * *struct *e *d *knowledge *: *| *concert *_ *s *inger *| *stadium * *: *stadium *_ *i *d * *, *location * *, *name * *, *capacity * *, *highest * *, *lowest * *, *average *| *singer * *: *singer *_ *i *d * *, *name * *, *country * *, *song *_ *name * *, *song *_ *release *_ *year * *, *age * *, *is *_ *male *| *concert * *: *concert *_ *i *d * *, *concert *_ *name * *, *theme * *, *stadium *_ *i *d * *, *year *| *singer *_ *in *_ *conce *r *t * *: *concert *_ *i *d * *, *singer *_ *i *d *</s>


trace_important_states_uskg.encoder:   0%|          | 0/1560 [00:00<?, ?it/s]

trace_important_states_uskg.decoder:   0%|          | 0/84 [00:00<?, ?it/s]

In [54]:
result.keys(), result['correct_prediction'], result['high_score'], result['low_score']

(dict_keys(['scores', 'low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'answer', 'window', 'correct_prediction', 'kind']),
 True,
 tensor(1.0000, device='cuda:0'),
 0.0005113474908284843)

In [ ]:
ctu.plot_trace_heatmap_t5(result)

#### Analysis sample 3 (ID = 97)

In [56]:
_id = 97
ex = processed_spider_dev[_id]
ex['question'], ex['seq_out']

('Find the model of the car whose weight is below the average weight.',
 'select t1.model from car_names as t1 join cars_data as t2 on t1.makeid = t2.id where t2.weight < (select avg(weight) from cars_data)')

In [57]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
dec_prompt = "select t1.model from car_names as t1 join cars_"
expect = "data"

In [58]:
struct_in

'| car_1 | continents : contid , continent | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year'

In [59]:
# text_in = ex['text_in']
# struct_in = ex['struct_in']

# enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
# dec_prompt = "select t1.model from car_names as t1 join cars_"
# expect = "data"

inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    enc_sentences=[enc_sentence],
    dec_prompts=[dec_prompt],
    device="cuda:0"
)
out = ctu.run_model_forward_uskg(mt_uskg.model, **inp)
logits = out["logits"][0, -1].detach().cpu().numpy()

top_5 = sorted(list(enumerate(logits)), key=lambda p: -p[1])[:5]

[(mt_uskg.tokenizer.decode([p[0]]), p[1]) for p in top_5]


[('data', -10.943039),
 ('stat', -21.073414),
 ('daten', -22.921432),
 ('re', -23.442766),
 ('performance', -23.719862)]

In [60]:
result = ctu.calculate_hidden_flow_uskg(
    mt_uskg,
    enc_sentence=enc_sentence,
    dec_prompt=dec_prompt,
    subject=None,
    replace=True,
    expect=expect,
)

calculate_hidden_flow_uskg(): corrupted input: *Find *the *model *of *the *car * *whose *weight *is *below *the *average *weight *. *; * *struct *e *d *knowledge *: *| *car *_ *1 *| *continent *s * *: *cont *i *d * *, *continent *| *countries * *: *country *i *d * *, *country *name * *, *continent *| *car *_ *makers * *: * *i *d * *, *maker * *, *full *name * *, *country *| *model *_ *list * *: *model *i *d * *, *maker * *, *model *| *car *_ *name *s * *: *make *i *d * *, *model * *, *make *| *cars *_ *data * *: * *i *d * *, * *mp *g * *, * *cylinder *s * *, * *e *disp *l * *, *horsepower * *, *weight * *, *accelerate * *, *year *</s>


trace_important_states_uskg.encoder:   0%|          | 0/1668 [00:00<?, ?it/s]

trace_important_states_uskg.decoder:   0%|          | 0/216 [00:00<?, ?it/s]

In [61]:
result.keys(), result['correct_prediction'], result['high_score'], result['low_score']

(dict_keys(['scores', 'low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'answer', 'window', 'correct_prediction', 'kind']),
 True,
 tensor(0.9999, device='cuda:0'),
 0.0008115010568872094)

In [ ]:
ctu.plot_trace_heatmap_t5(result)

In [63]:
# Another position

In [69]:
enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
dec_prompt = "select t1.model from car_names"
expect = "as"  # 't1' -> '_', 't', '1'

In [70]:
result = ctu.calculate_hidden_flow_uskg(
    mt_uskg,
    enc_sentence=enc_sentence,
    dec_prompt=dec_prompt,
    subject=None,
    replace=True,
    expect=expect,
)

calculate_hidden_flow_uskg(): corrupted input: *Find *the *model *of *the *car * *whose *weight *is *below *the *average *weight *. *; * *struct *e *d *knowledge *: *| *car *_ *1 *| *continent *s * *: *cont *i *d * *, *continent *| *countries * *: *country *i *d * *, *country *name * *, *continent *| *car *_ *makers * *: * *i *d * *, *maker * *, *full *name * *, *country *| *model *_ *list * *: *model *i *d * *, *maker * *, *model *| *car *_ *name *s * *: *make *i *d * *, *model * *, *make *| *cars *_ *data * *: * *i *d * *, * *mp *g * *, * *cylinder *s * *, * *e *disp *l * *, *horsepower * *, *weight * *, *accelerate * *, *year *</s>


trace_important_states_uskg.encoder:   0%|          | 0/1668 [00:00<?, ?it/s]

trace_important_states_uskg.decoder:   0%|          | 0/132 [00:00<?, ?it/s]

In [71]:
result.keys(), result['correct_prediction'], result['high_score'], result['low_score']

(dict_keys(['scores', 'low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'answer', 'window', 'correct_prediction', 'kind']),
 True,
 tensor(1.0000, device='cuda:0'),
 0.9942196011543274)

In [ ]:
ctu.plot_trace_heatmap_t5(result)

### Helpers

#### Evaluator

In [24]:
table_path = '/home/yshao/Projects/language/language/xsp/data/spider/tables.json'
db_dir = '/home/yshao/Projects/language/language/xsp/data/spider/database'

In [25]:
kmaps = sp_eval.build_foreign_key_map_from_json(table_path)
evaluator = sp_eval.Evaluator(db_dir=db_dir, kmaps=kmaps, etype='all')

In [26]:
# test
_sql_str = 'select t1.birth_date from people as t1 join poker_player as t2 on t1.people_id = t2.people_id order by t2.earnings asc limit 1'
db_name = 'poker_player'
schema = evaluator.schemas[db_name]
_sql = sp_eval.get_sql(schema, _sql_str)
sp_eval.count_component1(_sql), sp_eval.count_component2(_sql), sp_eval.count_others(_sql), \
evaluator.eval_hardness(_sql)

(3, 0, 0, 'hard')

#### Hardness

#### Column role

In [133]:
def _detect_column_role(dec_prompt):
    role_keyword_pattern = r'\W(select|where|join|group by|having|order by)\W'
    all_kws = re.findall(role_keyword_pattern, ' ' + dec_prompt + ' ')
    assert len(all_kws) > 0, dec_prompt
    col_role_kw = all_kws[-1]
    return col_role_kw

### Exp0: Study the influence of corrupting a token

In [75]:
_id = 5
ex = processed_spider_dev[_id]
ex['question'], ex['seq_out']

('What is the average, minimum, and maximum age for all French singers?',
 "select avg(age), min(age), max(age) from singer where country = 'France'")

In [76]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"

# expect = 'singer'
# expect = 'age'
expect = 'country'
dec_prompt = ctu.make_dec_prompt(ex['seq_out'], expect)

ex['enc_sentence'] = enc_sentence
ex['dec_prompt'] = dec_prompt
ex['expect'] = expect
dec_prompt, expect

('select avg(age), min(age), max(age) from singer where', 'country')

In [77]:
ans_toks = decode_tokens(mt_uskg.tokenizer, mt_uskg.tokenizer.encode(expect, add_special_tokens=False))
ans_toks

['country']

In [78]:
inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    [enc_sentence],
    [dec_prompt],
    answer=expect,
    device='cuda'
)

answer_len = len(mt_uskg.tokenizer.tokenize(expect))
with torch.no_grad():
    answers_t, base_score = [d[0] for d in ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)]
base_score = base_score.min().item()
answer = ctu.decode_sentences(mt_uskg.tokenizer, answers_t)
base_score, answer

(0.9999990463256836, 'country')

In [79]:
answers_t

tensor([684], device='cuda:0')

In [80]:
result_d = ctu.token_corruption_influence_uskg(
    mt_uskg,
#     enc_sentence=enc_sentence,
#     dec_prompt=dec_prompt,
#     expect=expect,
    ex,
    replace=True,
    use_tqdm=True,
    skips=('token',)
)

Corrupt effect: columns:   0%|          | 0/17 [00:00<?, ?it/s]

Corrupt effect: tables:   0%|          | 0/4 [00:00<?, ?it/s]

In [81]:
result_d.keys()

dict_keys(['enc_sentence', 'dec_prompt', 'expect', 'base_score', 'answers_t', 'answer', 'res_list'])

In [82]:
result = result_d['res_list']
result

[{'corrpt_type': 'column',
  'corrpt_idx': (38, 39),
  'corrpt_token': 'location',
  'corrpt_score': 0.9999988675117493,
  'corrpt_drop': 1.7881393432617188e-07},
 {'corrpt_type': 'column',
  'corrpt_idx': (44, 45),
  'corrpt_token': 'capacity',
  'corrpt_score': 0.9999990463256836,
  'corrpt_drop': 0.0},
 {'corrpt_type': 'column',
  'corrpt_idx': (47, 48),
  'corrpt_token': 'highest',
  'corrpt_score': 0.9999989867210388,
  'corrpt_drop': 5.960464477539063e-08},
 {'corrpt_type': 'column',
  'corrpt_idx': (50, 51),
  'corrpt_token': 'lowest',
  'corrpt_score': 0.9999990463256836,
  'corrpt_drop': 0.0},
 {'corrpt_type': 'column',
  'corrpt_idx': (53, 54),
  'corrpt_token': 'average',
  'corrpt_score': 0.9999991655349731,
  'corrpt_drop': -1.1920928955078125e-07},
 {'corrpt_type': 'column',
  'corrpt_idx': (67, 68),
  'corrpt_token': 'country',
  'corrpt_score': 4.299651834571705e-07,
  'corrpt_drop': 0.9999986163605001},
 {'corrpt_type': 'column',
  'corrpt_idx': (70, 73),
  'corrpt_tok

In [ ]:
[d for d in result if d['corrpt_type'] != 'token']

In [45]:
# ID = 503

_seq_len = len([d for d in result if d['corrpt_type'] == 'token'])
_tags = [[False, False, False] for _ in range(_seq_len)]  # (is_span_start, is_span_end, is_unit)
for i, d in enumerate(result):
    if d['corrpt_drop'] > 0.5:
        s, e = d['corrpt_idx']
        if e - s > 1:
            _tags[s][0] = True
            _tags[e-1][1] = True
        else:
            _tags[s][2] = True

l = []
for i in range(_seq_len):
    id_str = str(i)
    if _tags[i][2]:
        id_str = f'*{i}*'
    if _tags[i][0]:
        id_str = 'S' + id_str
    if _tags[i][1]:
        id_str = id_str + 'E'
    
    d = result[i]
    l.append(f'{id_str}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
#         l.append(f'{i}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
print('\n'.join(l))

0	How	0.0037964582443237305
1	many	0.0013628602027893066
2	battle	0.01413428783416748
3	s	-0.0008431673049926758
4	did	0.052848875522613525
5	not	0.09322965145111084
6	lose	0.002241969108581543
7	any	0.056729793548583984
8	ship	-0.0024902820587158203
9	with	0.04180067777633667
10	to	0.006940901279449463
11	n	0.07953697443008423
12	nage	0.0018101334571838379
13		0.0008842945098876953
14	'	0.0007312297821044922
15	225	-0.002007007598876953
16	'	0.000843048095703125
17	?	0.001425027847290039
18	;	-0.0008304119110107422
19		0.0033646225929260254
20	struct	0.005487203598022461
21	e	-0.0010203719139099121
22	d	0.0009642243385314941
23	knowledge	-0.00048089027404785156
24	:	-0.00015664100646972656
25	|	0.0022062063217163086
26	battle	0.0007958412170410156
27	_	0.000949561595916748
28	de	0.002287924289703369
29	a	0.0008778572082519531
30	th	0.0004265308380126953
31	|	0.0012375712394714355
32	battle	0.0064389705657958984
33		-0.0003082752227783203
34	:	-0.0015889406204223633
35		-0.002041161060

#### Exp3 (moved to py script)

In [84]:
res_dir = '/home/yshao/Projects/rome/results/exp3_relational_nodes_mutual'
os.makedirs(res_dir, exist_ok=True)
res_path = os.path.join(res_dir, 'exp=3.0_dev_column.jsonl')

In [ ]:
# all_results = []
# subject_type = 'column'
# remove_struct_duplicate_cols = True
# skips = ('token',)

# f = open(res_path, 'w')

# for ex_id, ex in enumerate(tqdm(processed_spider_dev)):
#     text_in = ex['text_in']
#     struct_in = ex['struct_in']

#     enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
#     ex['enc_sentence'] = enc_sentence
    
#     parsed_struct_in = ctu.parse_struct_in(struct_in)
#     col2table = defaultdict(list)
#     db_id_t, tables = parsed_struct_in
#     for table_name_t, cols in tables:
#         for col_name_t, vals in cols:
#             _, table_name, _ = table_name_t
#             _, col_name, _ = col_name_t
#             col2table[col_name].append(table_name)

#     token_ranges_dict = ctu.find_struct_name_ranges(mt_uskg.tokenizer, ex)
#     if subject_type == 'column':
#         node_name_ranges = token_ranges_dict['col_name_ranges']
#     elif subject_type == 'table':
#         node_name_ranges = token_ranges_dict['table_name_ranges']
    
#     sql_tokens = ctu.separate_punct(ex['seq_out']).split(' ')
#     sql_nodes = set()
#     for t in sql_tokens:
#         if t in node_name_ranges:
#             sql_nodes.add(t)
    
#     if subject_type == 'column':
#         for t in list(sql_nodes):
#             if len(col2table[t]) == 0:
#                 raise ValueError(struct_in, t)
#             elif (len(col2table[t]) > 1) and remove_struct_duplicate_cols:
#                 sql_nodes.remove(t)

#     for node in sql_nodes:
#         _ex = dict(ex)
        
#         tok_ranges = node_name_ranges[node]
        
#         expect = node
#         dec_prompt = ctu.make_dec_prompt(ex['seq_out'], expect)

#         ex['dec_prompt'] = dec_prompt
#         ex['expect'] = expect
        
#         # Check base performance 
#         inp = ctu.make_inputs_t5(
#             mt_uskg.tokenizer,
#             [enc_sentence],
#             [dec_prompt],
#             answer=expect,
#             device='cuda'
#         )

#         answer_len = len(mt_uskg.tokenizer.tokenize(expect))
#         with torch.no_grad():
#             answers_t, base_score = [d[0] for d in ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)]
#         base_score = base_score.min().item()
#         answer = ctu.decode_sentences(mt_uskg.tokenizer, answers_t)

#         result_d = ctu.token_corruption_influence_uskg(
#             mt_uskg,
#             # enc_sentence=enc_sentence,
#             # dec_prompt=dec_prompt,
#             # expect=expect,
#             ex,
#             replace=True,
#             use_tqdm=False,
#         )
        
#         result_d['ex_id'] = ex_id
#         all_results.append(result_d)
#         f.write(json.dumps(result_d, indent=None) + '\n')


#### Load results

In [106]:
with open(res_path, 'r') as f:
    all_results = [json.loads(l) for l in f]
len(all_results)

1655

In [107]:
all_results[0].keys(), all_results[0]['res_list'][0].keys()

(dict_keys(['enc_sentence', 'dec_prompt', 'expect', 'base_score', 'answers_t', 'answer', 'res_list', 'ex_id', 'correct_prediction', 'expect_table']),
 dict_keys(['corrpt_type', 'corrpt_idx', 'corrpt_token', 'corrpt_score', 'corrpt_drop']))

In [105]:
# # One-time patches
# # patch 1, for those d without "correct_prediction", set it True 
# # patch 2, for each d, add "expect_table" which is table of "expect" column 

# in_path = os.path.join(res_dir, 'exp=3.0_dev_column_old.jsonl')
# out_path = os.path.join(res_dir, 'exp=3.0_dev_column.jsonl')

# with open(in_path, 'r') as f:
#     all_results = [json.loads(l) for l in f]

# for d in all_results:
#     spider_ex = processed_spider_dev[d['ex_id']]
#     db_id = spider_ex['db_id']
#     col2table = db_col2table_cache[db_id]
#     d['expect_table'] = col2table[d['expect']][0]

# with open(out_path, 'w') as f:
#     for d in all_results:
#         f.write(json.dumps(d, indent=None) + '\n')
        

In [126]:
db_col2table_cache = dict()   # db_id -> col2table

scores_per_rel = {
    k: [] 
    for k in ['self', 'self_table', 'other_col', 'other_table', 'other_col_max', 'other_table_max']}
n_corr_pred = 0

for d in all_results:
    if not d['correct_prediction']:
        continue
        
    n_corr_pred += 1
    ex_id = d['ex_id']
    spider_ex = processed_spider_dev[ex_id]
    db_id = spider_ex['db_id']
    
    # TODO: make this a function: get_col2table(struct_in)
    if db_col2table_cache.get(db_id) is None:
        struct_in = spider_ex['struct_in']
        parsed_struct_in = ctu.parse_struct_in(struct_in)
        # parsed_struct_in_cache[db_id] = parsed_struct_in
    
        col2table = defaultdict(list)
        db_id_t, tables = parsed_struct_in
        for table_name_t, cols in tables:
            for col_name_t, vals in cols:
                _, table_name, _ = table_name_t
                _, col_name, _ = col_name_t
                col2table[col_name].append(table_name)
        db_col2table_cache[db_id] = col2table
    col2table = db_col2table_cache[db_id]
    
    col = d['expect']
    tab = d['expect_table']
    
    d['self_drop'] = None
    d['self_table_drop'] = None
    d['other_col_drop'] = []
    d['other_table_drop'] = []
    for res in d['res_list']:
        _drop = res['corrpt_drop']
        if res['corrpt_type'] == 'column':
            if res['corrpt_token'] == col:
                d['self_drop'] = _drop
            else:
                d['other_col_drop'].append(_drop)
        elif res['corrpt_type'] == 'table':
            if res['corrpt_token'] in col2table[col]:
                d['self_table_drop'] = _drop
            else:
                d['other_table_drop'].append(_drop)
    d['other_col_max_drop'] = max(d['other_col_drop'])
    d['other_table_max_drop'] = max(d['other_table_drop'])
    
    scores_per_rel['self'].append(d['self_drop'])
    scores_per_rel['self_table'].append(d['self_table_drop'])
    scores_per_rel['other_col'].extend(d['other_col_drop'])
    scores_per_rel['other_table'].extend(d['other_table_drop'])
    scores_per_rel['other_col_max'].append(d['other_col_max_drop'])
    scores_per_rel['other_table_max'].append(d['other_table_max_drop'])
    
[(k, len(scores_per_rel[k])) for k in scores_per_rel]

[('self', 1178),
 ('self_table', 1178),
 ('other_col', 18891),
 ('other_table', 3972),
 ('other_col_max', 1178),
 ('other_table_max', 1178)]

In [127]:
[(k, np.mean(scores_per_rel[k])) for k in scores_per_rel]

[('self', 0.671617640890816),
 ('self_table', 0.06442814806196535),
 ('other_col', 0.003760848689351455),
 ('other_table', 0.009762111044622013),
 ('other_col_max', 0.04336059879598554),
 ('other_table_max', 0.03053987597881895)]

In [128]:
self_table_effective_ids = []

for i, d in enumerate(all_results):
    self_table_drop = d.get('self_table_drop', 0.0)
    if self_table_drop > 0.5:
        self_table_effective_ids.append(i)
len(self_table_effective_ids)

78

In [129]:
print(self_table_effective_ids)

[155, 157, 180, 183, 202, 205, 210, 285, 355, 358, 369, 372, 373, 403, 406, 409, 412, 415, 455, 483, 607, 609, 650, 676, 678, 761, 950, 952, 953, 1016, 1044, 1138, 1140, 1144, 1147, 1164, 1167, 1187, 1189, 1191, 1193, 1194, 1195, 1196, 1198, 1201, 1203, 1205, 1207, 1210, 1213, 1221, 1224, 1228, 1236, 1239, 1242, 1254, 1255, 1258, 1259, 1263, 1267, 1271, 1275, 1279, 1293, 1296, 1299, 1303, 1309, 1313, 1323, 1325, 1456, 1485, 1521, 1523]


In [130]:
# Any type of samples make self-table effective? More with JOIN 

for i in self_table_effective_ids:
    res_d = all_results[i]
    print(i, res_d['dec_prompt'], '-->', res_d['expect'])


155 select t2. --> make
157 select t2.make, t1.year from cars_data as t1 join car_names as t2 on t1.id = t2. --> makeid
180 select t1.accelerate from cars_data as t1 join car_names as t2 on t1.id = t2. --> makeid
183 select t1.accelerate from cars_data as t1 join car_names as t2 on t1.id = t2. --> makeid
202 select min(weight) from cars_data where --> cylinders
205 select min(weight) from cars_data where --> cylinders
210 select t1.countryname, t1. --> countryid
285 select count(*) from countries as t1 join car_makers as t2 on t1.countryid = t2. --> country
355 select count(*) from flights as t1 join airports as t2 on t1. --> sourceairport
358 select count(*) from flights as t1 join airports as t2 on t1. --> sourceairport
369 select count(*) from flights as t1 join airports as t2 on t1.destairport = t2.airportcode join airports as t3 on t1. --> sourceairport
372 select count(*) from flights as t1 join airports as t2 on t1.destairport = t2.airportcode join airports as t3 on t1.sourceair

In [ ]:
_id = 157
res_d = all_results[_id]
result = res_d['res_list']
# _seq_len = max([d['corrpt_idx'][1] - 1 for d in result])
# _tags = [[False, False, False] for _ in range(_seq_len)]  # (is_span_start, is_span_end, is_unit)
# for i, d in enumerate(result):
#     if d['corrpt_drop'] > 0.5:
#         s, e = d['corrpt_idx']
#         if e - s > 1:
#             _tags[s][0] = True
#             _tags[e-1][1] = True
#         else:
#             _tags[s][2] = True

# spider_ex = processed_spider_dev[res_d['ex_id']]
print(res_d['enc_sentence'])
print(res_d['dec_prompt'], '-->', res_d['expect'])
print()

l = []
for d in result:
    _prefix = ''
    db_id = spider_ex['db_id']
    
    if d["corrpt_type"] == 'column' and d["corrpt_token"] == res_d['expect']:
        _prefix = '**'
    elif d["corrpt_type"] == 'table' and d["corrpt_token"] == res_d['expect_table']:
        _prefix = '*>'
    l.append(f'{_prefix}{d["corrpt_type"]}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
#         l.append(f'{i}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
print('\n'.join(l))

#### Split by different aspects

In [134]:
## Hardness

samples_by_hardness = defaultdict(list)

for res_d in all_results:
    if not res_d['correct_prediction']:
        continue
    hardness = spider_id2hardness[res_d['ex_id']]
    samples_by_hardness[hardness].append(res_d)

[(h, len(samples)) for h, samples in samples_by_hardness.items()]

[('medium', 565), ('easy', 219), ('hard', 216), ('extra', 178)]

In [ ]:
res_d

In [136]:
for h in ['easy', 'medium', 'hard', 'extra']:
    samples = samples_by_hardness[h]
    print(f'Hardness: {h} ({len(samples)})')
    # print('[Example sql]', samples[0]['seq_out'])
    
    for rel in scores_per_rel.keys():
        _rel_k = rel + '_drop'
        rel_scores = [x for res_d in samples for x in ctu.ensure_list(res_d[_rel_k])]
        avg_score = np.mean(rel_scores)
        print(f'{rel}\t{avg_score:.4f}\t')
    print()

Hardness: easy (219)
self	0.6729	
self_table	0.0200	
other_col	0.0022	
other_table	0.0024	
other_col_max	0.0389	
other_table_max	0.0063	

Hardness: medium (565)
self	0.6568	
self_table	0.0510	
other_col	0.0028	
other_table	0.0060	
other_col_max	0.0344	
other_table_max	0.0185	

Hardness: hard (216)
self	0.6767	
self_table	0.0705	
other_col	0.0069	
other_table	0.0177	
other_col_max	0.0619	
other_table_max	0.0551	

Hardness: extra (178)
self	0.7109	
self_table	0.1544	
other_col	0.0047	
other_table	0.0192	
other_col_max	0.0546	
other_table_max	0.0688	



In [137]:
## Column role

samples_by_column_role = defaultdict(list)

for res_d in all_results:
    if not res_d['correct_prediction']:
        continue
    col_role_kw = _detect_column_role(res_d['dec_prompt'])
    samples_by_column_role[col_role_kw].append(res_d)

[(r, len(samples)) for r, samples in samples_by_column_role.items()]

[('select', 669),
 ('where', 346),
 ('order by', 68),
 ('join', 90),
 ('group by', 1),
 ('having', 4)]

In [138]:
for r, samples in samples_by_column_role.items():
    if len(samples) < 10:
        continue
    print(f'Column role: {r} ({len(samples)})')
    # print('[Example sql]', samples[0]['seq_out'])
    
    for rel in scores_per_rel.keys():
        _rel_k = rel + '_drop'
        rel_scores = [x for res_d in samples for x in ctu.ensure_list(res_d[_rel_k])]
        avg_score = np.mean(rel_scores)
        print(f'{rel}\t{avg_score:.4f}\t')
    print()

Column role: select (669)
self	0.6103	
self_table	0.0210	
other_col	0.0025	
other_table	0.0036	
other_col_max	0.0383	
other_table_max	0.0131	

Column role: where (346)
self	0.7348	
self_table	0.0284	
other_col	0.0017	
other_table	0.0022	
other_col_max	0.0186	
other_table_max	0.0074	

Column role: order by (68)
self	0.8414	
self_table	0.0353	
other_col	0.0011	
other_table	0.0027	
other_col_max	0.0106	
other_table_max	0.0061	

Column role: join (90)
self	0.7382	
self_table	0.5404	
other_col	0.0241	
other_table	0.0809	
other_col_max	0.2031	
other_table_max	0.2695	



In [139]:
## Text match 

samples_by_text_match = {k: [] for k in ['exact', 'partial', 'no-match']}

for res_d in all_results:
    if not res_d['correct_prediction']:
        continue
    spider_ex = processed_spider_dev[res_d['ex_id']]
    text_match = ctu.check_text_match(spider_ex, col=res_d['expect'], tab=res_d['expect_table'])
    samples_by_text_match[text_match].append(res_d)

[(m, len(samples)) for m, samples in samples_by_text_match.items()]

[('exact', 644), ('partial', 173), ('no-match', 361)]

In [140]:
for m, samples in samples_by_text_match.items():
    print(f'Text-match: {m} ({len(samples)})')
    # print('[Example sql]', samples[0]['seq_out'])
    
    for rel in scores_per_rel.keys():
        _rel_k = rel + '_drop'
        rel_scores = [x for res_d in samples for x in ctu.ensure_list(res_d[_rel_k])]
        avg_score = np.mean(rel_scores)
        print(f'{rel}\t{avg_score:.4f}\t')
    print()

Text-match: exact (644)
self	0.5317	
self_table	0.0215	
other_col	0.0020	
other_table	0.0034	
other_col_max	0.0223	
other_table_max	0.0106	

Text-match: partial (173)
self	0.7947	
self_table	0.0020	
other_col	0.0020	
other_table	0.0022	
other_col_max	0.0457	
other_table_max	0.0112	

Text-match: no-match (361)
self	0.8622	
self_table	0.1710	
other_col	0.0079	
other_table	0.0258	
other_col_max	0.0798	
other_table_max	0.0754	



### Exp-1: column/table corruption
- In py script

#### Plotting results

In [310]:
exp_name = 'dev_column'

res_json_path = f'/home/yshao/Projects/rome/results/struct_node_restore/{exp_name}.jsonl'
with open(res_json_path, 'r') as f:
#     res_dicts = [json.loads(l) for l in f if l]
    all_str = f.read()
all_str = all_str.replace('{"ex_id":', '\n{"ex_id":').strip()
res_dicts = [json.loads(l) for l in all_str.split('\n')]
len(res_dicts)

154

In [311]:
len([r for d in res_dicts for r in d['trace_results']])

147

In [312]:
good_trace_results = []
for d in res_dicts:
    for r in d['trace_results']:
        if r['correct_prediction'] and r['is_good_sample']:
            r['ex_id'] = d['ex_id']
            good_trace_results.append(r)
len(good_trace_results)

92

In [313]:
d.keys(), good_trace_results[0].keys(), type(good_trace_results[0]['scores'])

(dict_keys(['ex_id', 'trace_results']),
 dict_keys(['low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'subject_range_individual_indices', 'answer', 'window', 'correct_prediction', 'kind', 'scores', 'is_good_sample', 'target_node', 'ex_id']),
 list)

In [ ]:
# Generating all plots
fig_save_dir = f'/home/yshao/Projects/rome/results/struct_node_restore/figs/{exp_name}'

for i, r in enumerate(tqdm(good_trace_results)):
    result = dict(r)

    enc_s, dec_s = result['scores']
    enc_s = np.array(enc_s)
    dec_s = np.array(dec_s)
    result['scores'] = [enc_s, dec_s]

    ex_id = r['ex_id']
    ctu.plot_trace_heatmap_t5(result, savepdf=os.path.join(fig_save_dir, f'{i}-ex_id={ex_id}.pdf'))
    

#### Finding special samples

In [314]:
for i, r in enumerate(good_trace_results):
    _, dec_s = r['scores']
    ## dec_s: (n_toks, n_layers)
    non_last_token_s = np.array(dec_s)[:-1]
    if (non_last_token_s > 0.5).any():
        print(f'{i}-ex_id={r["ex_id"]}')
    

57-ex_id=69
60-ex_id=70
83-ex_id=139
87-ex_id=143
88-ex_id=144
89-ex_id=145
91-ex_id=153


### Exp-1.1: severing decoder cross-attention
- TODO: merge to py script

#### Single sample (ID = 2)

In [ ]:
_id = 2
ex = processed_spider_dev[_id]
ex['question'], ex['struct_in'], ex['seq_out']

In [ ]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
enc_tokenized = mt_uskg.tokenizer(enc_sentence)

token_ranges_dict = ctu.find_struct_name_ranges(mt_uskg.tokenizer, enc_tokenized['input_ids'], struct_in)
token_ranges_dict

In [ ]:
col = 'name'
tok_ranges = token_ranges_dict['col_name_ranges'][col]
tok_indices = [i for s, e in tok_ranges for i in range(s, e)]

dec_prompt = make_dec_prompt(ex['seq_out'], col)

dec_prompt, tok_indices

In [ ]:
result = ctu.calculate_hidden_flow_uskg(
    mt_uskg,
    enc_sentence=enc_sentence,
    dec_prompt=dec_prompt,
    expect=col,
    e_range=tok_indices,
    enc_token_range=[],    # no analysis
    dec_token_range=None,  # full analysis
    tokens_to_mix_individual_indices=True,
    replace=True,
    sever_kind='self_attn'
)

In [ ]:
# no sever
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [ ]:
# sever cross_attn
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [ ]:
# sever mlp; a bit better than no severing??
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [ ]:
# sever self_attn
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [338]:
len(result['scores'][1][-1])

24

### Exp-2: dirty text recovery
- Corrupt the text, restore different parts of encoder final output
- Idea is to check the existence of contextual understanding (incorporating text info into struct representation)
- (In py script)

#### Single sample
- ID = 2, col = 'name': (name exact match with text)
    - none: wrong; text: correct; struct: correct; col: correct
- ID = 5, col = 'country': (no exact match, but value match with text (French) )
    - none: wrong; text: correct; struct: correct; col: correct
- ID = 145, col = 'year': (no exact match, but value match with text (1980) )
    - none: wrong; text: correct; struct: correct; col: correct
- ID = 7, col = 'song_release_year': (multi-token, and with confusing columns in table (song_name) )
    - none: wrong; text: wrong; struct: correct; col: wrong
    - hypothesis: need clean struct to avoid confusion
- ID = 185, col = 'airportcode': (multi-token, with confusing col (airportname), but have match with text (airport code) )
    - none: wrong; text: correct; struct: wrong (2nd token); col: wrong
    - differs from hypothesis... clean struct should know sql info about the column

In [192]:
_id = 24
ex = processed_spider_dev[_id]
ex['db_id'], ex['question'], ex['struct_in'], ex['seq_out']

('concert_singer',
 'Show the stadium name and capacity with most number of concerts in year 2014 or after.',
 '| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id',
 'select t2.name, t2.capacity from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id where t1.year >= 2014 group by t2.stadium_id order by count(*) desc limit 1')

In [193]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
enc_tokenized = mt_uskg.tokenizer(enc_sentence)

text_range, struct_range = ctu.find_text_struct_in_range(mt_uskg.tokenizer, enc_tokenized['input_ids'])
text_range, struct_range

((0, 17), (24, 134))

In [34]:
token_ranges_dict = ctu.find_struct_name_ranges(mt_uskg.tokenizer, enc_tokenized['input_ids'], struct_in)

col_name_ranges = token_ranges_dict['col_name_ranges']

sql_tokens = ctu.separate_punct(ex['seq_out']).split(' ')

sql_cols = set()
for t in sql_tokens:
    if t in col_name_ranges:
        sql_cols.add(t)
sql_cols

{'name', 'nationality'}

In [35]:
parsed_struct_in = ctu.parse_struct_in(struct_in)
parsed_struct_in

((1, 'orchestra', 'orchestra'),
 [((3, 'conductor', 'conductor'),
   [[(5, 'conductor_id', 'conductor_id'), []],
    [(7, 'name', 'name'), []],
    [(9, 'age', 'age'), []],
    [(11, 'nationality', 'nationality ( USA )'), [(13, 'USA', 'USA')]],
    [(16, 'year_of_work', 'year_of_work'), []]]),
  ((18, 'orchestra', 'orchestra'),
   [[(20, 'orchestra_id', 'orchestra_id'), []],
    [(22, 'orchestra', 'orchestra'), []],
    [(24, 'conductor_id', 'conductor_id'), []],
    [(26, 'record_company', 'record_company'), []],
    [(28, 'year_of_founded', 'year_of_founded'), []],
    [(30, 'major_record_format', 'major_record_format'), []]]),
  ((32, 'performance', 'performance'),
   [[(34, 'performance_id', 'performance_id'), []],
    [(36, 'orchestra_id', 'orchestra_id'), []],
    [(38, 'type', 'type'), []],
    [(40, 'date', 'date'), []],
    [(42, 'official_ratings_(millions)', 'official_ratings_(millions)'), []],
    [(44, 'weekly_rank', 'weekly_rank'), []],
    [(46, 'share', 'share'), []]]),

In [37]:
col = 'nationality'
# tok_ranges = token_ranges_dict['col_name_ranges'][col]
# tok_indices = [i for s, e in tok_ranges for i in range(s, e)]
dec_prompt = ctu.make_dec_prompt(ex['seq_out'], col)
expect = col
dec_prompt, expect

('select name from conductor where', 'nationality')

In [38]:
col_toks = token_ranges_dict['col_name_ranges'][col]
col, col_toks

('nationality', [(45, 51)])

In [39]:
# result = ctu.calculate_hidden_flow_uskg(
#     mt_uskg,
#     enc_sentence=enc_sentence,
#     dec_prompt=dec_prompt,
#     expect=col,
#     e_range=text_range,
#     enc_token_range=[],    # no analysis
#     dec_token_range=None,  # full analysis
#     tokens_to_mix_individual_indices=False,
#     replace=True,
#     sever_kind=None,
# )

inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    [enc_sentence] * 11,
    [dec_prompt] * 11,
    answer=expect)

encoder_text_last_layer_states = [
    (tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
    for tnum in range(*text_range)
]

encoder_struct_last_layer_states = [
    (tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
    for tnum in range(*struct_range)
]

encoder_col_last_layer_states = [
    (tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
    for tnum in col_toks
]

answer_len = len(mt_uskg.tokenizer.tokenize(expect))
answers_t, base_score = [d[0] for d in ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)]

In [40]:
base_score

tensor([1.0000, 1.0000], device='cuda:0')

In [41]:
## Corrupting text: expect wrong pred 

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=[],
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(0.1734, device='cuda:0'), tensor(1.0000, device='cuda:0')],
 tensor(0.1734, device='cuda:0'))

In [42]:
## Restoring text encoding for decoder, but struct encoding are with dirty text encoding 

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=encoder_text_last_layer_states,
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(0.9999, device='cuda:0')],
 tensor(0.9999, device='cuda:0'))

In [43]:
## Restoring clean struct encoding but dirty text encoding for decoder
## Prediction being correct means encoder final output has "contextual" or "semantic" understanding of struct_in 

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=encoder_struct_last_layer_states,
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(1., device='cuda:0')],
 tensor(1.0000, device='cuda:0'))

In [44]:
## Restoring clean col_name encoding but dirty text encoding for decoder (stricter than above)

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=encoder_col_last_layer_states,
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(1.0000, device='cuda:0')],
 tensor(1.0000, device='cuda:0'))

In [45]:
## For exp-2.1: mutual corruption
# First pass: corrupt text (no restore)
# Second pass: corrupt struct, no restore, reset struct output to first pass

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=[],
    states_to_unpatch=encoder_struct_last_layer_states,
    answers_t=answers_t,
    tokens_to_mix_1st_pass=text_range,
    tokens_to_mix=struct_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(1.0000, device='cuda:0')],
 tensor(1.0000, device='cuda:0'))

#### Load & Check results

In [175]:
res_path = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/exp=2_train_column.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

7000

In [176]:
all_samples[5439]['ex_id'], all_samples[5440]['ex_id'], all_samples[-1]['ex_id'], 

(5439, 5440, 6999)

In [181]:
total_samples = 0
n_good_samples = 0
n_too_hard = 0      # wrong answer 
n_too_easy = 0      # base - low < 0.5

base_scores = []
low_scores = []

## len = n_good_samples
restore_scores_dict = {
    'text': [],
    'struct': [],
    'node': [],
    'struct_no_node': [],
    'node_corrupt_all': [],
    # 2.0.1: cancelled
    # 'ctname': [],      # col name + table name (col belongs to) 
    # 'catname': [],     # col name + all table names 
    # 'full_table': [],  # full table (col belongs to) 
    # 'all_col': [],     # all col names (regardless of table)
}

## len = total_samples
mutual_scores_dict = {
    f'{text}-{struct}': []
    for text in ['clean_t', 'dc_t', 'dirty_t']
    for struct in ['clean_s', 'dc_s', 'dirty_s']
}


In [182]:
good_samples = []

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        total_samples += 1
        if d['is_good_sample']:
            n_good_samples += 1
            d['ex_id'] = i
            good_samples.append(d)
        elif not d['correct_prediction']:
            n_too_hard += 1
        else:
            assert d['base_score'] - d['low_score'] < 0.5
            n_too_easy += 1
            
total_samples, (n_good_samples, len(good_samples)), n_too_hard, n_too_easy

(10390, (6177, 6177), 2568, 1645)

In [183]:
d.keys(), d['mutual_scores'].keys()

(dict_keys(['text_in', 'struct_in', 'seq_out', 'dec_prompt', 'expect', 'table', 'expect_input_range', 'mutual_scores', 'answer', 'answers_t', 'correct_prediction', 'base_score', 'low_score', 'is_good_sample', 'r_text_score', 'r_struct_score', 'r_node_score', 'r_struct_no_node_score', 'r_node_corrupt_all_score', 'ex_id']),
 dict_keys(['clean_t-clean_s', 'clean_t-dc_s', 'clean_t-dirty_s', 'dc_t-clean_s', 'dc_t-dc_s', 'dc_t-dirty_s', 'dirty_t-clean_s', 'dirty_t-dc_s', 'dirty_t-dirty_s']))

In [184]:
for d in good_samples:
    base_scores.append(d['base_score'])
    low_scores.append(d['low_score'])
    for k in restore_scores_dict.keys():
        restore_scores_dict[k].append(d[f'r_{k}_score'])
    for k in mutual_scores_dict.keys():
        mutual_scores_dict[k].append(d['mutual_scores'][k])

In [185]:
len(base_scores), len(restore_scores_dict['text']), len(mutual_scores_dict['dc_t-dc_s'])

(6177, 6177, 6177)

In [187]:
# results for exp2
for k, scores in restore_scores_dict.items():
    avg = np.mean(scores)
    print(f'{k}\t{avg:.4f}')

text	0.6394
struct	0.8891
node	0.7598
struct_no_node	0.1105
node_corrupt_all	0.7477


In [188]:
print(f'Score\tavg_gain\tperc_recover')
for score_label, high_scores in [
    ('base_scores', base_scores),
    ('restore_text_scores', restore_scores_dict['text']),
    ('restore_struct_scores', restore_scores_dict['struct']),
    ('restore_node_scores', restore_scores_dict['node']),
    ('restore_struct_no_node_scores', restore_scores_dict['struct_no_node']),
    ('restore_node_corrupt_all_scores', restore_scores_dict['node_corrupt_all']),
]:
    avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
    perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
    print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')

Score	avg_gain	perc_recover
base_scores	0.9613	1.0000
restore_text_scores	0.6008	0.6291
restore_struct_scores	0.8504	0.9021
restore_node_scores	0.7211	0.7764
restore_struct_no_node_scores	0.0718	0.0792
restore_node_corrupt_all_scores	0.7091	0.7505


In [189]:
# results for exp2.1.1 
msg = ' '*8
for k_t in ['clean_t', 'dc_t', 'dirty_t']:
    msg += f'{k_t:8s}'
msg += '\n'
for k_s in ['clean_s', 'dc_s', 'dirty_s']:
    msg += f'{k_s:8s}'
    for k_t in ['clean_t', 'dc_t', 'dirty_t']:
        k = f'{k_t}-{k_s}'
        scores = mutual_scores_dict[k]
        avg = np.mean(scores)
        msg += f'{avg:.4f}  '
    msg += '\n'
print(msg)

        clean_t dc_t    dirty_t 
clean_s 1.0000  0.9896  0.8891  
dc_s    0.6394  0.4793  0.0387  
dirty_s 0.3766  0.3141  0.0037  



#### Split by hardness

In [118]:
# build an index for hardness 

spider_indices_per_hardness = defaultdict(list)
spider_id2hardness = dict()

for i, ex in enumerate(processed_spider_dev):
    db_id = ex['db_id']
    sql_str = ex['seq_out']
    sql = sp_eval.get_sql(evaluator.schemas[db_id], sql_str)
    hardness = evaluator.eval_hardness(sql)
    spider_indices_per_hardness[hardness].append(i)
    spider_id2hardness[i] = hardness

[(h, len(indices)) for h, indices in spider_indices_per_hardness.items()]

[('easy', 248), ('medium', 446), ('hard', 174), ('extra', 166)]

In [119]:
spider_indices_per_hardness['extra'][:10]

[24, 25, 41, 42, 57, 58, 59, 60, 61, 62]

In [164]:
# exp2 + exp2.1.1 
# res_path = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/exp=2_dev_column.jsonl'

# with open(res_path, 'r') as f:
#     all_samples = [json.loads(l) for l in f]
# len(all_samples)

In [165]:
good_samples = []

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        if d['is_good_sample']:
            d['ex_id'] = i
            good_samples.append(d)
len(good_samples)

983

In [166]:
samples_by_hardness = defaultdict(list)

# for ex in all_samples:
#     for d in ex['trace_results']:
#         if not d['is_good_sample']:
#             continue

for d in good_samples:
#     sql_str = d['seq_out']
#     db_id = d['struct_in'].split('|')[1].strip()    # TODO: add db_id to result during experiment main run 
#     sql = sp_eval.get_sql(evaluator.schemas[db_id], sql_str)
#     hardness = evaluator.eval_hardness(sql)
    hardness = spider_id2hardness[d['ex_id']]
    samples_by_hardness[hardness].append(d)

In [167]:
[(h, len(samples)) for h, samples in samples_by_hardness.items()]

[('medium', 480), ('easy', 181), ('hard', 186), ('extra', 136)]

In [177]:
for h in ['easy', 'medium', 'hard', 'extra']:
    samples = samples_by_hardness[h]
    print(f'Hardness: {h} ({len(samples)})')
    print('[Example sql]', samples[0]['seq_out'])
    print(f'Score\tavg_gain\tperc_recover')
    
    low_scores = [d['low_score'] for d in samples]
    for score_label in [
        'base_score',
        'r_text_score',
        'r_struct_score',
        'r_col_score',
        'r_struct_no_col_score',
        'r_col_corrupt_all_score'
    ]:
        high_scores = [d[score_label] for d in samples]
        avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
        perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
        print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')
    print()

Hardness: easy (181)
[Example sql] select distinct country from singer where age > 20
Score	avg_gain	perc_recover
base_score	0.9572	1.0000
r_text_score	0.6232	0.6740
r_struct_score	0.8541	0.9061
r_col_score	0.7442	0.8011
r_struct_no_col_score	0.0945	0.1105
r_col_corrupt_all_score	0.7708	0.8177

Hardness: medium (480)
[Example sql] select name, country, age from singer order by age desc
Score	avg_gain	perc_recover
base_score	0.9462	1.0000
r_text_score	0.5521	0.5938
r_struct_score	0.8132	0.8688
r_col_score	0.6674	0.7188
r_struct_no_col_score	0.0577	0.0583
r_col_corrupt_all_score	0.6774	0.7208

Hardness: hard (186)
[Example sql] select song_name from singer where age > (select avg(age) from singer)
Score	avg_gain	perc_recover
base_score	0.9462	1.0000
r_text_score	0.5662	0.5914
r_struct_score	0.7303	0.7688
r_col_score	0.6793	0.7204
r_struct_no_col_score	0.0947	0.1183
r_col_corrupt_all_score	0.7154	0.7419

Hardness: extra (136)
[Example sql] select t2.name, t2.location from concert as t1 jo

#### Split by column role

In [169]:
role_keyword_pattern = r'\W(select|where|join|group by|having|order by)\W'

In [170]:
_sql_str = samples_by_hardness['extra'][0]['seq_out']
_sql_str

'select t2.name, t2.location from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id where t1.year = 2014 intersect select t2.name, t2.location from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id where t1.year = 2015'

In [171]:
re.findall(role_keyword_pattern, ' ' + _sql_str + ' ')

['select', 'join', 'where', 'select', 'join', 'where']

In [173]:
samples_by_column_role = defaultdict(list)

for d in good_samples:
#     all_kws = re.findall(role_keyword_pattern, ' ' + d['dec_prompt'] + ' ')
#     assert len(all_kws) > 0, d['dec_prompt']
#     col_role_kw = all_kws[-1]
    col_role_kw = _detect_column_role(d['dec_prompt'])
    samples_by_column_role[col_role_kw].append(d)

[(r, len(samples)) for r, samples in samples_by_column_role.items()]

[('select', 632),
 ('where', 231),
 ('order by', 57),
 ('join', 60),
 ('group by', 1),
 ('having', 2)]

In [176]:
for r in samples_by_column_role.keys():
    samples = samples_by_column_role[r]
    if len(samples) < 10:
        continue
    print(f'Column role: {r} ({len(samples)})')
    print('[Example prompt]', samples[0]['dec_prompt'])
    print(f'Score\tavg_gain\tperc_recover')
    
    low_scores = [d['low_score'] for d in samples]
    for score_label in [
        'base_score',
        'r_text_score',
        'r_struct_score',
        'r_col_score',
        'r_struct_no_col_score',
        'r_col_corrupt_all_score'
    ]:
        high_scores = [d[score_label] for d in samples]
        avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
        perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
        print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')
    print()

Column role: select (632)
[Example prompt] select name, country,
Score	avg_gain	perc_recover
base_score	0.9639	1.0000
r_text_score	0.6161	0.6519
r_struct_score	0.7748	0.8070
r_col_score	0.6741	0.7184
r_struct_no_col_score	0.0500	0.0475
r_col_corrupt_all_score	0.6990	0.7278

Column role: where (231)
[Example prompt] select avg(age), min(age), max(age) from singer where
Score	avg_gain	perc_recover
base_score	0.9186	1.0000
r_text_score	0.5183	0.5584
r_struct_score	0.8727	0.9654
r_col_score	0.8211	0.9091
r_struct_no_col_score	0.0841	0.1212
r_col_corrupt_all_score	0.7374	0.8139

Column role: order by (57)
[Example prompt] select song_name, song_release_year from singer order by
Score	avg_gain	perc_recover
base_score	0.9447	1.0000
r_text_score	0.4849	0.5263
r_struct_score	0.9077	0.9649
r_col_score	0.8048	0.8596
r_struct_no_col_score	-0.0102	0.0000
r_col_corrupt_all_score	0.5500	0.5789

Column role: join (60)
[Example prompt] select t2.countryname from car_makers as t1 join countries as t2 on

In [35]:
# res_path = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/exp=2_dev_column.jsonl'
# with open(res_path, 'r') as f:
#     all_samples = [json.loads(l) for l in f]
# len(all_samples)

#### Split by text match

In [182]:
d = good_samples[244]
spider_ex = processed_spider_dev[d['ex_id']]
col = d['expect']
tab = d['table']
d['table'], d['expect'], d['text_in'], ctu.check_text_match(spider_ex, col, tab)

('shop',
 'number_products',
 'Sort all the shops by number products in descending order, and return the name, location and district of each shop.',
 'exact')

In [148]:
samples_by_text_match = {k: [] for k in ['exact', 'partial', 'no-match']}

for d in good_samples:
    spider_ex = processed_spider_dev[d['ex_id']]
    text_match = _check_text_match(spider_ex, d)
    samples_by_text_match[text_match].append(d)

[(m, len(samples)) for m, samples in samples_by_text_match.items()]

[('exact', 553), ('partial', 159), ('no-match', 271)]

In [154]:
for m in ['exact', 'partial', 'no-match']:
    samples = samples_by_text_match[m]
    print(f'Text match: {m} ({len(samples)})')
    print('[Example]', samples[0]['text_in'])
    print(f"[Example] {samples[0]['dec_prompt']} *{samples[0]['expect']}*")
    print(f'Score\tavg_gain\tperc_recover')
    
    low_scores = [d['low_score'] for d in samples]
    for score_label in [
        'base_score',
        'r_text_score',
        'r_struct_score',
        'r_col_score',
        'r_struct_no_col_score',
        'r_col_corrupt_all_score'
#         'm-dc_t-dc_s-score',
    ]:
        high_scores = [d[score_label] for d in samples]
        avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
        perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
        print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')
    print()

Text match: exact (553)
[Example] Show name, country, age for all singers ordered by age from the oldest to the youngest.
[Example] select name, country, *age*
Score	avg_gain	perc_recover
base_score	0.9501	1.0000
r_text_score	0.7393	0.7830
r_struct_score	0.7989	0.8517
r_col_score	0.6998	0.7468
r_struct_no_col_score	0.0679	0.0723
r_col_corrupt_all_score	0.7193	0.7595

Text match: partial (159)
[Example] Show the name and the release year of the song by the youngest singer.
[Example] select song_name, *song_release_year*
Score	avg_gain	perc_recover
base_score	0.9513	1.0000
r_text_score	0.2488	0.2767
r_struct_score	0.7862	0.8239
r_col_score	0.6924	0.7484
r_struct_no_col_score	0.0420	0.0692
r_col_corrupt_all_score	0.5945	0.6415

Text match: no-match (271)
[Example] What is the average, minimum, and maximum age of all singers from France?
[Example] select avg(age), min(age), max(age) from singer where *country*
Score	avg_gain	perc_recover
base_score	0.9366	1.0000
r_text_score	0.3370	0.3469


In [200]:
ex = samples_by_text_match['no-match'][50]

In [202]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in} The database is as following: {struct_in}"

f"{enc_sentence} => {ex['seq_out']}"

"What are the names of all European countries with at least 3 manufacturers? The database is as following: | car_1 | continents : contid , continent ( europe ) | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year => select t1.countryname from countries as t1 join continents as t2 on t1.continent = t2.contid join car_makers as t3 on t1.countryid = t3.country where t2.continent = 'europe' group by t1.countryname having count(*) >= 3;"

In [203]:
ex['ex_id']

130

### Exp-2.2: dirty text struct restore

#### Load & Plotting results

In [217]:
res_json_path = f'/home/yshao/Projects/rome/results/exp2.2_dirty_text_struct_restore/exp=2.2_dev_column.jsonl'

with open(res_json_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]

good_trace_results = []
for ex in all_samples:
    for r in ex['trace_results']:
        if r['is_good_sample']:
            r['ex_id'] = ex['ex_id']
            good_trace_results.append(r)
    
len(all_samples), len(good_trace_results)

(1034, 983)

In [ ]:
good_trace_results[0]

In [ ]:
# # Generating all plots
# fig_save_dir = f'/home/yshao/Projects/rome/results/figs/exp2.2_dirty_text_struct_restore/exp=2.2_dev_column'

# for i, r in enumerate(tqdm(good_trace_results)):
#     result = dict(r)

#     enc_s, dec_s = result['scores']
    
#     # enc_s = np.array(enc_s)
#     enc_layers_vec = np.array(enc_s[0])
#     assert enc_layers_vec.shape == (mt_uskg.num_enc_layers,)
#     enc_token_range, = result['enc_token_range']
    
#     enc_s = np.zeros((len(result['input_tokens']), mt_uskg.num_enc_layers))
#     enc_s[enc_token_range] = enc_layers_vec
    
#     dec_s = np.zeros((len(result['dec_input_tokens']), mt_uskg.num_dec_layers))
#     result['scores'] = [enc_s, dec_s]

#     ex_id = r['ex_id']
#     ctu.plot_trace_heatmap_t5(result, savepdf=os.path.join(fig_save_dir, f'{i}-ex_id={ex_id}.pdf'))
    

#### Check avg per layer

In [ ]:
scores_per_layer = [[] for _ in range(mt_uskg.num_enc_layers)]

for r in good_trace_results:
    enc_s, dec_s = r['scores']
    for l, s in enumerate(enc_s[0]):
        scores_per_layer[l].append(s)

In [ ]:
avg_by_layer = [np.mean(scores) for scores in scores_per_layer]
avg_by_layer

In [ ]:
print('Layer\tAvg. Score')
for l, avg in enumerate(avg_by_layer):
    print(f'{l:<5d}\t{avg:.4f}')

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(avg_by_layer, label='All')
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

####  by hardness 

In [ ]:
# {h : [[score], ...]}
scores_by_hardness = {hardness : [] for hardness in ['easy', 'medium', 'hard', 'extra']}   

for r in good_trace_results:
    enc_s, dec_s = r['scores']
    scores, = enc_s
    hardness = spider_id2hardness[r['ex_id']]
    scores_by_hardness[hardness].append(scores)

[(h, len(scores)) for h, scores in scores_by_hardness.items()]

In [ ]:
for h, sample_scores in scores_by_hardness.items():
    print(h)
    scores_by_layer = zip(*sample_scores)
    avg_by_layer = [np.mean(scores) for scores in scores_by_layer]
    print([f'{avg:.2f}' for avg in avg_by_layer])
    print()

In [ ]:
fig, ax = plt.subplots()
ax.plot(avg_by_layer, label='All')
for h, sample_scores in scores_by_hardness.items():
    h_scores_by_layer = zip(*sample_scores)
    h_avg_by_layer = [np.mean(scores) for scores in h_scores_by_layer]
    ax.plot(h_avg_by_layer, label=h)
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

####  by column role 

In [ ]:
# {h : [[score], ...]}
scores_by_column_role = defaultdict(list)

for r in good_trace_results:
    enc_s, dec_s = r['scores']
    scores, = enc_s
    role = _detect_column_role(' '.join(r['dec_input_tokens']))
    scores_by_column_role[role].append(scores)

[(r, len(scores)) for r, scores in scores_by_column_role.items()]

In [ ]:
for r, sample_scores in scores_by_column_role.items():
    if len(sample_scores) < 10:
        continue
    print(r)
    scores_by_layer = zip(*sample_scores)
    avg_by_layer = [np.mean(scores) for scores in scores_by_layer]
    print([f'{avg:.2f}' for avg in avg_by_layer])
    print()

In [ ]:
fig, ax = plt.subplots()
ax.plot(avg_by_layer, label='All')
for r, sample_scores in scores_by_column_role.items():
    if len(sample_scores) < 10:
        continue
    r_scores_by_layer = zip(*sample_scores)
    r_avg_by_layer = [np.mean(scores) for scores in r_scores_by_layer]
    ax.plot(r_avg_by_layer, label=r)
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

#### by text matching

In [ ]:
scores_by_text_matching = {k: [] for k in ['exact', 'partial', 'no-match']}

for r in good_trace_results:
    spider_ex = processed_spider_dev[r['ex_id']]
    col = r['target_node']
    tab = r['target_node_table']
    m = ctu.check_text_match(spider_ex, col, tab)

    enc_s, dec_s = r['scores']
    scores, = enc_s
    scores_by_text_matching[m].append(scores)

[(m, len(samples)) for m, samples in scores_by_text_matching.items()]

In [ ]:
for r, sample_scores in scores_by_text_matching.items():
    if len(sample_scores) < 10:
        continue
    print(r)
    scores_by_layer = zip(*sample_scores)
    avg_by_layer = [np.mean(scores) for scores in scores_by_layer]
    print([f'{avg:.2f}' for avg in avg_by_layer])
    print()

In [ ]:
fig, ax = plt.subplots()
ax.plot(avg_by_layer, label='All')
for m, sample_scores in scores_by_text_matching.items():
    if len(sample_scores) < 10:
        continue
    r_scores_by_layer = zip(*sample_scores)
    r_avg_by_layer = [np.mean(scores) for scores in r_scores_by_layer]
    ax.plot(r_avg_by_layer, label=m)
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

## USKG error analysis

In [335]:
# adapted from play_pred()

def pred_sql(mt, ex):
    text_in = ex['text_in']
    struct_in = ex['struct_in']

    txt = f"{text_in}; structed knowledge: {struct_in}"
    
    tokenized_txt = mt.tokenizer_uskg([txt], max_length=1024, padding="max_length", truncation=True)
    
    device = mt.model.device
    pred = mt.tokenizer_uskg.batch_decode(
      mt.model.generate(
        torch.tensor(tokenized_txt.data['input_ids'], dtype=int, device=device),
        torch.tensor(tokenized_txt.data['attention_mask'], dtype=int, device=device),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
    )
    return pred[0]


In [336]:
# adapted from evaluator.evaluate_one()

def evaluate_sql(evaluator, db_name, gold, predicted):
    schema = evaluator.schemas[db_name]
    g_sql = sp_eval.get_sql(schema, gold)
    hardness = evaluator.eval_hardness(g_sql)
    # self.scores[hardness]["count"] += 1
    # self.scores["all"]["count"] += 1

    parse_error = False
    try:
        p_sql = sp_eval.get_sql(schema, predicted)
    except:
        # If p_sql is not valid, then we will use an empty sql to evaluate with the correct sql
        p_sql = {
            "except": None,
            "from": {"conds": [], "table_units": []},
            "groupBy": [],
            "having": [],
            "intersect": None,
            "limit": None,
            "orderBy": [],
            "select": [False, []],
            "union": None,
            "where": [],
        }

        # TODO fix
        parse_error = True

    # rebuild sql for value evaluation
    kmap = evaluator.kmaps[db_name]
    g_valid_col_units = sp_eval.build_valid_col_units(g_sql["from"]["table_units"], schema)
    g_sql = sp_eval.rebuild_sql_val(g_sql)
    g_sql = sp_eval.rebuild_sql_col(g_valid_col_units, g_sql, kmap)
    p_valid_col_units = sp_eval.build_valid_col_units(p_sql["from"]["table_units"], schema)
    p_sql = sp_eval.rebuild_sql_val(p_sql)
    p_sql = sp_eval.rebuild_sql_col(p_valid_col_units, p_sql, kmap)
    
    exec_score = None
    partial_scores = None
    exact_score = None
    if evaluator.etype in ["all", "exec"]:
        exec_score = sp_eval.eval_exec_match(
            evaluator.db_paths[db_name], predicted, gold, p_sql, g_sql
        )
        exec_score = int(exec_score)
    if evaluator.etype in ["all", "match"]:
        partial_scores = evaluator.eval_partial_match(p_sql, g_sql)
        exact_score = evaluator.eval_exact_match(p_sql, g_sql, partial_scores)
        # update_scores_match(self.scores, exact_score, hardness, partial_scores, PARTIAL_TYPES)

    return {
        "predicted": predicted,
        "gold": gold,
        "predicted_parse_error": parse_error,
        "hardness": hardness,
        "exact": exact_score,
        "partial": partial_scores,
        "exec": exec_score,
    }

In [354]:
db_dir = '/home/yshao/Projects/language/language/xsp/data/spider/database'

def execute_sql(db, sql_str):
    db_path = os.path.join(db_dir, db, f'{db}.sqlite')
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    try:
        cursor.execute(sql_str)
        res = cursor.fetchall()
    except:
        res = 'ERROR'
    conn.close()
    return res

In [403]:
ex = processed_spider_dev[503]
ex.keys()

dict_keys(['query', 'question', 'db_id', 'db_path', 'db_table_names', 'db_column_names', 'db_column_types', 'db_primary_keys', 'db_foreign_keys', 'rat_sql_graph', 'serialized_schema', 'struct_in', 'text_in', 'seq_out'])

In [404]:
pred = pred_sql(mt_uskg, ex)
ex['db_id'], ex['text_in'], ex['seq_out'], pred

('battle_death',
 "How many battles did not lose any ship with tonnage '225'?",
 "select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage = '225' );",
 'select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage > 225 )')

In [ ]:
text_in = ex['text_in']
struct_in = ex['struct_in']
txt = f"{text_in}; structed knowledge: {struct_in}"
txt_toks = mt_uskg.tokenizer_uskg.tokenize(txt)
len(txt_toks)

In [ ]:
txt_toks

In [405]:
exec_res = execute_sql(ex['db_id'], ex['seq_out'])
ex['seq_out'], exec_res

("select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage = '225' );",
 [(7,)])

In [406]:
exec_res = execute_sql(ex['db_id'], pred)
pred, exec_res

('select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage > 225 )',
 [(3,)])

In [407]:
chatgpt_pred = """
SELECT COUNT(DISTINCT battle.id) AS num_battles
FROM battle
LEFT JOIN ship ON battle.id = ship.lost_in_battle
WHERE (ship.tonnage != '225' OR ship.tonnage IS NULL)
"""

execute_sql(ex['db_id'], chatgpt_pred)

[(8,)]

In [339]:
eval_res = evaluate_sql(evaluator, db_name=spider_ex['db_id'], gold=spider_ex['seq_out'], predicted=pred)

In [340]:
eval_res

{'predicted': 'select avg(num_of_staff) from museum where open_year  < 2009',
 'gold': 'select avg(num_of_staff) from museum where open_year < 2009',
 'predicted_parse_error': False,
 'hardness': 'easy',
 'exact': True,
 'partial': {'select': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 1,
   'pred_total': 1},
  'select(no AGG)': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 1,
   'pred_total': 1},
  'where': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 1, 'pred_total': 1},
  'where(no OP)': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 1,
   'pred_total': 1},
  'group(no Having)': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 0,
   'pred_total': 0},
  'group': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 0, 'pred_total': 0},
  'order': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 0, 'pred_total': 0},
  'and/or': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 1, 'pred_total': 1},
  'IUEN': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 0, 'pred_total': 0},


In [411]:
# Eval all 
# TODO: rerun with tokenizer_uskg decoding 

eval_sql_results = []

for ex_id, ex in enumerate(tqdm(processed_spider_dev)):
    pred = pred_sql(mt_uskg, ex)
    eval_res = evaluate_sql(evaluator, db_name=ex['db_id'], gold=ex['seq_out'], predicted=pred)
    eval_sql_results.append(eval_res)
    

  0%|          | 0/1034 [00:00<?, ?it/s]

In [412]:
avg_exact = np.mean([d['exact'] for d in eval_sql_results])
avg_exec = np.mean([d['exec'] for d in eval_sql_results])
avg_exact, avg_exec

(0.6692456479690522, 0.6808510638297872)

In [ ]:
for i, d in enumerate(eval_sql_results):
    if d['exact'] and d['exec']:
        continue
    err_msg = ('A' if not d['exact'] else '') + ('X' if not d['exec'] else '')
    ex = processed_spider_dev[i]
    print(f'ID = {i}: {err_msg}  ({ex["db_id"]}) {ex["text_in"]}')
    print(f'Pred: {d["predicted"]}')
    print(f'Gold: {d["gold"]}')
    print()

In [ ]:
evaluator.schemas['dog_kennels'].schema

In [414]:
# acc by db_id 
eval_sql_results_by_db_id = defaultdict(list)

for i, d in enumerate(eval_sql_results):
    d['ex_id'] = i
    ex = processed_spider_dev[i]
    db_id = ex['db_id']
    eval_sql_results_by_db_id[db_id].append(d)

len(eval_sql_results_by_db_id)

20

In [415]:
for db_id, results in eval_sql_results_by_db_id.items():
    _avg_exact = np.mean([d['exact'] for d in results])
    _avg_exec = np.mean([d['exec'] for d in results])
    print(f'{db_id}\t{_avg_exact:.4f}\t{_avg_exec:.4f}')

concert_singer	0.8889	0.8889
pets_1	0.5714	0.7381
car_1	0.3478	0.3913
flight_2	0.7000	0.7500
employee_hire_evaluation	0.9474	0.9737
cre_Doc_Template_Mgt	0.8333	0.9048
course_teach	0.8667	0.9333
museum_visit	0.7222	0.8333
wta_1	0.6774	0.6129
battle_death	0.5000	0.5000
student_transcripts_tracking	0.6667	0.6795
tvshow	0.7258	0.6613
poker_player	0.8750	0.8750
voter_1	0.6000	0.6667
world_1	0.5083	0.4833
orchestra	0.8000	0.8750
network_1	0.6250	0.4643
dog_kennels	0.5854	0.5976
singer	0.8667	0.8667
real_estate_properties	0.5000	0.5000


## Temp

### Debugging exp

In [497]:
ex = processed_spider_dev[97]
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
dec_prompt = "select t1.model from"
expect = "car_names"

In [498]:
inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    enc_sentences=[enc_sentence]*11,
    dec_prompts=[dec_prompt]*11,
    answer=expect
)

In [499]:
ans_toks = decode_tokens(mt_uskg.tokenizer, mt_uskg.tokenizer.encode(expect, add_special_tokens=False))
ans_toks

['car', '_', 'name', 's']

In [500]:
mt_uskg.tokenizer.decode(inp['decoder_input_ids'][0])

'<pad> select t1.model from car_name'

In [501]:
mt_uskg.tokenizer.tokenize(dec_prompt), len(mt_uskg.tokenizer.tokenize(dec_prompt))

(['▁select', '▁', 't', '1.', 'model', '▁from'], 6)

In [502]:
len(inp['input_ids'][0])

139

In [503]:
# r = ctu.trace_with_patch_uskg_multi_token(
#     mt_uskg.model,
#     inp=inp,
# #     states_to_patch=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', 3))],
#     states_to_patch=[],
# #     answers_t=mt_uskg.tokenizer.encode(expect, add_special_tokens=False),
# #     tokens_to_mix=(0, len(inp['input_ids'][0])-1),
#     tokens_to_mix=None,
#     replace=True,
# )

In [504]:
# r

[tensor(0.5643, device='cuda:0'),
 tensor(1.0000, device='cuda:0'),
 tensor(0.9996, device='cuda:0'),
 tensor(1.0000, device='cuda:0')]

In [506]:
answer_len = len(mt_uskg.tokenizer.tokenize(expect))
pred_out = ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)
pred, p = pred_out
pred[0]

tensor([ 443,  834, 4350,    7], device='cuda:0')

In [224]:
# pred_toks = decode_tokens(mt_uskg.tokenizer, pred[0])
# list(zip(pred_toks, ans_toks))

[('model', 'car'), ('_', '_'), ('name', 'name'), ('s', 's')]

In [507]:
ex['db_id'], ex['question'], ex['struct_in'], ex['seq_out']

('car_1',
 'Find the model of the car whose weight is below the average weight.',
 '| car_1 | continents : contid , continent | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year',
 'select t1.model from car_names as t1 join cars_data as t2 on t1.makeid = t2.id where t2.weight < (select avg(weight) from cars_data)')

In [ ]:
r = ctu.trace_with_repatch_uskg_multi_token(
    mt_uskg.model,
    inp=inp,
    states_to_patch=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', 3))],
    states_to_unpatch=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', 4, 'cross_attn'))],
    states_to_patch_1st_pass=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', l)) for l in range(mt_uskg.num_enc_layers)],
    answers_t=pred[0],
    tokens_to_mix=(10, 20, 30, 40),
    tokens_to_mix_1st_pass=(5, 15, 25),
    tokens_to_mix_individual_indices=True,
    replace=True,
)

In [511]:
r

[tensor(0.5877, device='cuda:0'),
 tensor(1., device='cuda:0'),
 tensor(0.9988, device='cuda:0'),
 tensor(1.0000, device='cuda:0')]

In [ ]:
# # inp = ctu.make_inputs_t5(mt_uskg.tokenizer, [text_in] * 11, [dec_prompt] * 11)
# inp = ctu.make_inputs_t5(mt_uskg.tokenizer, [enc_sentence] * 11, [dec_prompt] * 11, answer=expect, device='cpu')
# # answer_t, base_score = [d[0] for d in ctu.predict_from_input_uskg(mt_uskg.model, inp)]
# # base_score = base_score.item()
# # [answer] = ctu.decode_tokens(mt_uskg.tokenizer, [answer_t])
# answer_len = 1
# if expect is not None:
#     answer_len = len(mt_uskg.tokenizer.tokenize(expect))
# with torch.no_grad():
#     answers_t, base_score = [d[0] for d in ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)]
# # base_score = base_score.min().item()
# # [answer] = decode_tokens(mt.tokenizer, [answer_t])
# answer = ctu.decode_sentences(mt_uskg.tokenizer, answers_t)

# expect, answers_t, answer, base_score

In [ ]:
# # e_range = (129, 132)
# # e_range = [8, 13, 131]
# e_range = list(range(7, 14))

# r = ctu.trace_with_patch_uskg(
#     mt_uskg.model,
#     inp=inp,
#     states_to_patch=[], 
#     answers_t=answer_t, 
#     tokens_to_mix=e_range,
#     tokens_to_mix_individual_indices=True,
#     replace=True,
# )

# r

### Moving models

In [82]:
mt_uskg.model.device

device(type='cpu')

### tokenizer

In [11]:
tokenizer = mt_uskg.tokenizer

In [433]:
s = "mary: has a little  lambbb"
s_ = "mary: has a little lambbb"
tokenizer.tokenize(s), tokenizer.tokenize(s_)

(['▁mar', 'y', ':', '▁has', '▁', 'a', '▁little', '▁lamb', 'b', 'b'],
 ['▁mar', 'y', ':', '▁has', '▁', 'a', '▁little', '▁lamb', 'b', 'b'])

In [428]:
t = tokenizer(s)
t

{'input_ids': [3157, 63, 10, 65, 3, 9, 385, 17871, 115, 115, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [429]:
list(enumerate(t.tokens()))

[(0, '▁mar'),
 (1, 'y'),
 (2, ':'),
 (3, '▁has'),
 (4, '▁'),
 (5, 'a'),
 (6, '▁little'),
 (7, '▁lamb'),
 (8, 'b'),
 (9, 'b'),
 (10, '</s>')]

In [430]:
t.word_to_tokens(4)

TokenSpan(start=7, end=10)

In [431]:
ctu.decode_sentences(tokenizer, t['input_ids'][7:10])

'lambbb'

In [ ]:
"(a(a)a)".rindex(")"), "(a(a)a)".index(")")

### Test struct_in parsing

In [66]:
_struct_in = """| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , \
average | singer : singer_id , name ( First Last ) , country ( France , Germany , United States ) , \
song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , \
stadium_id , year ( 2008 , 2012 , 2022 ) | singer_in_concert : concert_id , singer_id"""

In [71]:
ctu.parse_struct_in(_struct_in)

[((3, 'stadium'),
  [[(5, 'stadium_id'), []],
   [(7, 'location'), []],
   [(9, 'name'), []],
   [(11, 'capacity'), []],
   [(13, 'highest'), []],
   [(15, 'lowest'), []],
   [(17, 'average'), []]]),
 ((19, 'singer'),
  [[(21, 'singer_id'), []],
   [(23, 'name'), [(25, 'First Last')]],
   [(29, 'country'), [(31, 'France'), (33, 'Germany'), (35, 'United States')]],
   [(39, 'song_name'), []],
   [(41, 'song_release_year'), []],
   [(43, 'age'), []],
   [(45, 'is_male'), []]]),
 ((47, 'concert'),
  [[(49, 'concert_id'), []],
   [(51, 'concert_name'), []],
   [(53, 'theme'), []],
   [(55, 'stadium_id'), []],
   [(57, 'year'), [(59, '2008'), (61, '2012'), (63, '2022')]]]),
 ((66, 'singer_in_concert'),
  [[(68, 'concert_id'), []], [(70, 'singer_id'), []]])]

In [84]:
_text_in = text_in

enc_sentence = f"{_text_in}; structed knowledge: {_struct_in}"
enc_tokenized = mt_uskg.tokenizer(enc_sentence)

In [85]:
token_ranges_dict = ctu.find_struct_name_ranges(mt_uskg.tokenizer, enc_tokenized['input_ids'], _struct_in)

In [86]:
token_ranges_dict

{'db_id_ranges': defaultdict(list, {'concert_singer': [(25, 29)]}),
 'table_name_ranges': defaultdict(list,
             {'stadium': [(30, 31)],
              'singer': [(56, 57)],
              'concert': [(106, 107)],
              'singer_in_concert': [(142, 149)]}),
 'col_name_ranges': defaultdict(list,
             {'stadium_id': [(33, 37), (123, 127)],
              'location': [(39, 40)],
              'name': [(42, 43), (65, 66)],
              'capacity': [(45, 46)],
              'highest': [(48, 49)],
              'lowest': [(51, 52)],
              'average': [(54, 55)],
              'singer_id': [(59, 63), (157, 161)],
              'country': [(73, 74)],
              'song_name': [(87, 90)],
              'song_release_year': [(92, 97)],
              'age': [(99, 100)],
              'is_male': [(102, 105)],
              'concert_id': [(109, 113), (151, 155)],
              'concert_name': [(115, 118)],
              'theme': [(120, 121)],
              'year': [(129

In [161]:
for d_key, d in token_ranges_dict.items():
    for name, ranges in d.items():
        for s, e in ranges:
            recs_name = ctu.decode_sentences(mt_uskg.tokenizer, enc_tokenized['input_ids'][s:e])
            print(f'{d_key}\t{name}\t{recs_name}')

db_id_ranges	concert_singer	concert_singer
table_name_ranges	stadium	stadium
table_name_ranges	singer	singer
table_name_ranges	concert	concert
table_name_ranges	singer_in_concert	singer_in_concert
col_name_ranges	stadium_id	stadium_id
col_name_ranges	stadium_id	stadium_id
col_name_ranges	location	location
col_name_ranges	name	name
col_name_ranges	name	name
col_name_ranges	capacity	capacity
col_name_ranges	highest	highest
col_name_ranges	lowest	lowest
col_name_ranges	average	average
col_name_ranges	singer_id	singer_id
col_name_ranges	singer_id	singer_id
col_name_ranges	country	country ( France )
col_name_ranges	song_name	song_name
col_name_ranges	song_release_year	song_release_year
col_name_ranges	age	age
col_name_ranges	is_male	is_male
col_name_ranges	concert_id	concert_id
col_name_ranges	concert_id	concert_id
col_name_ranges	concert_name	concert_name
col_name_ranges	theme	theme
col_name_ranges	year	year
val_name_ranges	France	France


### Stanza

In [99]:
import stanza

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

In [102]:
doc = nlp('cars data')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

word: cars 	lemma: car
word: data 	lemma: datum


In [122]:
doc = nlp('cars data')
doc.sentences[0].words[0]

{
  "id": 1,
  "text": "cars",
  "lemma": "car",
  "upos": "NOUN",
  "xpos": "NNS",
  "feats": "Number=Plur",
  "start_char": 0,
  "end_char": 4
}

In [ ]:
# lemmatizer = WordNetLemmatizer()

# stanza_nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

In [132]:
# lemm_cache = dict()

# def _lemmatize(name):
# #     toks = []
# #     for t in name.split('_'):
# #         lemm_t = lemm_cache.get(t, None)
# #         if lemm_t is None:
# #             _doc = stanza_nlp(t)
# #             lemm_t = _doc.sentences[0].words[0].lemma
# #             lemm_cache[t] = lemm_t
# #         toks.append(lemm_t)
# #     return '_'.join(toks)
#     _name = ' '.join(name.split('_'))
#     _doc = stanza_nlp(_name)
#     lemm_name = [w.lemma for sent in _doc.sentences for w in sent.words]
#     return lemm_name
    

In [ ]:
# _lemmatize('ranking_date')

In [130]:
# def _check_text_match(spider_ex, result_d):
#     col = result_d['expect']
#     tab = result_d['table']
#     node_name = f'<C>{tab}::{col}'
#     nodes = spider_ex['rat_sql_graph']['nodes']
    
#     if node_name in nodes:
#         node_idx = nodes.index(node_name)
#     else:
#         _col = _lemmatize(col)
#         _tab = _lemmatize(tab)
#         _node_name = f'<C>{_tab}::{_col}'
# #         if col != _col:
# #             print(f'* Lemmatize: {node_name} ==> {_node_name}')
#         if _node_name in nodes:
#             node_idx = nodes.index(_node_name)
#         else:
#             assert False, ((node_name, _node_name), nodes)
    
#     rel_matrix = json.loads(spider_ex['rat_sql_graph']['relations'])
#     rel_row = rel_matrix[node_idx]
    
#     if REL2ID['cqCEM'] in rel_row:
#         return 'exact'
#     elif REL2ID['cqCPM'] in rel_row:
#         return 'partial'
#     else:
#         return 'no-match'
    

### Tokenizer-fast / uskg

In [331]:
ex = processed_spider_dev[416]

In [332]:
# TODO: add special token (<, <=) to t5-tokenizer 
# could use uskg tokenizer, but it's not TokenizerFast 

text_in = ex['text_in']
struct_in = ex['struct_in']

txt = f"{text_in}; structed knowledge: {struct_in}"

tokenized_txt = mt_uskg.tokenizer([txt], max_length=1024, padding="max_length", truncation=True)

device = mt_uskg.model.device
_output = mt_uskg.model.generate(
    torch.tensor(tokenized_txt.data['input_ids'], dtype=int, device=device),
    torch.tensor(tokenized_txt.data['attention_mask'], dtype=int, device=device),
    num_beams=1, 
    max_length=256
)

_output

tensor([[    0,  1738,     3,     9,   208,   122,   599,  5525,   834,   858,
           834, 26416,    61,    45,  7071,   213,   539,   834,  1201, 32100,
          2464,     1]], device='cuda:0')

In [334]:
mt_uskg.tokenizer.batch_decode(
    _output,
    skip_special_tokens=True 
)

['select avg(num_of_staff) from museum where open_year 2009']

In [333]:
mt_uskg.tokenizer_uskg.batch_decode(
    _output,
    skip_special_tokens=True 
)

['select avg(num_of_staff) from museum where open_year  < 2009']

### Prepare input for chatgpt

In [401]:
ex = processed_spider_dev[503]

In [402]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"

print(f"{enc_sentence} => {ex['seq_out']}")

How many battles did not lose any ship with tonnage '225'?; structed knowledge: | battle_death | battle : id , name , date , bulgarian_commander , latin_commander , result | ship : lost_in_battle , id , name , tonnage , ship_type , location , disposition_of_ship | death : caused_by_ship_id , id , note , killed , injured => select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage = '225' );


### One-time patches

In [173]:
# # For exp2: ['trace_results'][.]: ['%col%_score'] -> ['%node%_score']

# res_dir = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/'
# in_path = os.path.join(res_dir, 'exp=2_train_column-tmp.jsonl')
# out_path = os.path.join(res_dir, 'exp=2_train_column.jsonl')

# with open(in_path, 'r') as f:
#     all_results = [json.loads(l) for l in f]

# for ex_d in all_results:
#     for d in ex_d['trace_results']:
#         if not d['is_good_sample']:
#             continue
#         d['r_node_score'] = d['r_col_score']
#         d['r_struct_no_node_score'] = d['r_struct_no_col_score']
#         d['r_node_corrupt_all_score'] = d['r_col_corrupt_all_score']
#         del d['r_col_score']
#         del d['r_struct_no_col_score']
#         del d['r_col_corrupt_all_score']

# all_results.sort(key=lambda d: d['ex_id'])
        
# with open(out_path, 'w') as f:
#     for d in all_results:
#         f.write(json.dumps(d, indent=None) + '\n')
        

In [ ]:
d

### other temp

In [147]:
t = torch.tensor(3, dtype=int)
t

tensor(3)

In [149]:
type(t.item())

int